# 学習方法の例

In [1]:
#!pip install pandas
import numpy as np
import pandas as pd
#!pip install matplotlib
import matplotlib.pyplot as plt
#!pip install scikit-learn
from sklearn.model_selection import train_test_split
import pickle
import sys, os
#import torch
sys.path.append(os.pardir) 
from collections import OrderedDict
from common.layers import Convolution, MaxPooling, ReLU, Affine, SoftmaxWithLoss, SGD, RMSProp, Adam, Adadelta, Dropout, MeanSquaredLoss, LeakyReLU, mean_squared_error, BatchNormalization

In [2]:
path = os.getcwd()
print(path)

/home/kota/Downloads/dl_online_2022_1_submit_katakana_kota_isayama_20220215_1


In [3]:
#! pip install torch
#!conda install torch
#!conda install -c pytorch pytorch
import torch
if torch.cuda.is_available():
    device = 'cuda'
else :
    device = 'cpu'
print(device)

ModuleNotFoundError: No module named 'torch'

In [4]:
#!pip install tensorflow
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2023-04-23 21:28:31.228084: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 21:28:31.783329: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-23 21:28:32.350928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-23 21:28:32.394967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3258023619576457061
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 22642753536
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13344350284389445685
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:09:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

## データを読む1

In [5]:
train_data = np.load("/home/kota/Downloads/dl_online_2022_1_submit_katakana_kota_isayama_20220215_1/1_data/new/train_data.npy")
#train_data = np.reshape(train_data, (50970, 1, 28, 28))
train_label = np.load("/home/kota/Downloads/dl_online_2022_1_submit_katakana_kota_isayama_20220215_1/1_data/new/train_label.npy")
#train_label = np.reshape(train_label, (9000, 2352))
print(train_data.shape)
print(train_label.shape)

(48521, 28, 28)
(48521, 15)


In [6]:
# 正規化
train_data =  (train_data - train_data.min()) / train_data.max()
#train_data = (train_data)/255
train_data = train_data.astype('float32')
#print(train_data)

In [7]:
# 配列形式変更
train_data = train_data.reshape(-1, 28*28)
#train_label = train_label.reshape(-1, 28*28)
print("train_data.shape=", train_data.shape)

train_data.shape= (48521, 784)


## trainとtestに分割する

In [8]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, 
                                                                                        test_size=0.3, random_state=1234,
                                                                                        shuffle=True
                                                                                       )
print(X_train.shape, X_test.shape)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()


train = X_train
test = X_test
#train = train.reshape(-1, 28*28)
#test = test.reshape(-1, 28*28)
#train_labels = lb.fit_transform(y_train)
#test_labels = lb.fit_transform(y_test)
train_labels = y_train
test_labels = y_test

#print(train_labels.shape)
#print(train_labels)

(33964, 784) (14557, 784)


In [9]:
train = train.reshape(-1, 1, 28, 28)
test = test.reshape(-1, 1, 28, 28)
#train_labels = train_labels.reshape(-1, 1, 28, 28)
#test_labels = test_labels.reshape(-1, 1, 28, 28)
#print(train, test)

In [10]:
class DeepConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        weight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLUを使う場合に推奨される初期値
       


        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = weight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = weight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = weight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)


        # レイヤの生成===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(ReLU())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(ReLU())
        self.layers.append(MaxPooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(ReLU())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(ReLU())
        self.layers.append(MaxPooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(ReLU())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(ReLU())
        self.layers.append(MaxPooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(ReLU())
        self.layers.append(Dropout(0.15))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        #self.layers.append(Dropout(0.25))
        
        self.last_layer = SoftmaxWithLoss()


    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x


    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)


    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)


        acc = 0.0


        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)


        return acc / x.shape[0]


    def gradient(self, x, t):
        # forward
        self.loss(x, t)


        # backward
        dout = 1
        dout = self.last_layer.backward(dout)


        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)


        # 設定
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db


        return grads






In [ ]:
x = train
t = train_labels
#t = np.reshape(t, (500, 15))
print(t.shape)

epochs = 10
batch_size = 256
lr = 0.01
rho = 0.9
#optimizer = SGD(lr=lr)
#optimizer = RMSProp(lr=0.0015, rho=0.9)
#optimizer = Adam(lr = 0.0015)
optimizer = Adadelta(rho = 0.9)
x = x.reshape(-1,1,28,28)
#t = t.reshape(-1, 1, 28, 28)
#print(t.shape)

#print(x.shape)

# 繰り返し回数
xsize = x.shape[0]
#print(xsize)
iter_num = np.ceil(xsize / batch_size).astype(np.int)

# 2層NNのオブジェクト生成
#tnet = TwoLayerNet(input_size=28*28, hidden_size=10, output_size=15)
dnet = DeepConvNet(input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=15)

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

#print("初期パラメータ")
#print(snet.params["W1"].round(4))
#print(snet.params["b1"].round(4))
#print(snet.params["W2"].round(4))
#print(snet.params["b2"].round(4))

for epoch in range(epochs):
    print("epoch=%s"%epoch)
    
    # シャッフル
    idx = np.arange(xsize)
    np.random.shuffle(idx)

    for it in range(iter_num):
        """
        ランダムなミニバッチを順番に取り出す
        """
        print("it=", it)
        mask = idx[batch_size*it : batch_size*(it+1)]
    
        # ミニバッチの生成
        x_train = x[mask]
        t_train = t[mask]
        #print(x_train.shape, t_train.shape)
        # 勾配の計算
        grads = dnet.gradient(x_train, t_train)
        #print(grads)
        #print(snet.params.shape)
        #print(lr)
        optimizer.update(dnet.params, grads)

        




    ## 学習経過の記録
    
    # 訓練データにおけるloss
    train_loss.append(dnet.loss(x,  t))
    
    # テストデータにおけるloss
    test_loss.append(dnet.loss(test, test_labels))
    
    # 訓練データにて精度を確認
    train_accuracy.append(dnet.accuracy(x, t))

    # テストデータにて精度を算出
    test_accuracy.append(dnet.accuracy(test, test_labels))

(33964, 15)
epoch=0
it= 0


/tmp/ipykernel_21475/2597010849.py:23: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  iter_num = np.ceil(xsize / batch_size).astype(np.int)


it= 1
it= 2
it= 3
it= 4
it= 5
it= 6
it= 7
it= 8
it= 9
it= 10
it= 11
it= 12
it= 13
it= 14
it= 15
it= 16
it= 17
it= 18
it= 19
it= 20
it= 21
it= 22
it= 23
it= 24
it= 25
it= 26
it= 27
it= 28
it= 29
it= 30
it= 31
it= 32
it= 33
it= 34
it= 35
it= 36
it= 37
it= 38
it= 39
it= 40
it= 41
it= 42
it= 43
it= 44
it= 45
it= 46
it= 47
it= 48
it= 49
it= 50
it= 51
it= 52
it= 53
it= 54
it= 55
it= 56
it= 57
it= 58
it= 59
it= 60
it= 61
it= 62
it= 63
it= 64
it= 65
it= 66
it= 67
it= 68
it= 69
it= 70
it= 71
it= 72
it= 73
it= 74
it= 75
it= 76
it= 77
it= 78
it= 79
it= 80
it= 81
it= 82
it= 83
it= 84
it= 85
it= 86
it= 87
it= 88
it= 89
it= 90
it= 91
it= 92
it= 93
it= 94
it= 95
it= 96
it= 97
it= 98
it= 99
it= 100
it= 101
it= 102
it= 103
it= 104
it= 105
it= 106
it= 107
it= 108
it= 109
it= 110
it= 111
it= 112
it= 113
it= 114
it= 115
it= 116
it= 117
it= 118
it= 119
it= 120
it= 121
it= 122
it= 123
it= 124
it= 125
it= 126
it= 127
it= 128
it= 129
it= 130
it= 131
it= 132


## 学習

In [ ]:
# lossとaccuracyのグラフ化
df_log = pd.DataFrame({"train_loss":train_loss,
             "test_loss":test_loss,
             "train_accuracy":train_accuracy,
             "test_accuracy":test_accuracy})

df_log.plot(style=['r-', 'r--', 'b-', 'b--'])
plt.ylim([0,7])
plt.ylabel("Accuracy or loss")
plt.xlabel("epochs")
plt.show()

## 学習済みモデルの出力

In [ ]:
with open("katakana_model.pickle", "wb") as f:
    pickle.dump(dnet, f)